## Import libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

import gc
import pickle
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import r2_score
from sklearn.model_selection import StratifiedKFold

## Prepare data for model training

In [2]:
with open("../input/he-ffi-preprocess-data-v2-2/HE_FFI_Dataset.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

del processed_data
gc.collect()

0

In [3]:
cat_cols = ['Insurance_company','expiry_dt_year','expiry_dt_quarter','Condition',
            'expiry_dt_month','expiry_dt_day_week','expiry_dt_day_weekend']

train_df[cat_cols] = train_df[cat_cols].astype(int)
test_df[cat_cols] = test_df[cat_cols].astype(int)
cat_cols_indices = [train_df.columns.get_loc(col) for col in cat_cols]
print(cat_cols_indices)

[0, 6, 7, 3, 8, 12, 13]


In [4]:
Xtrain = train_df.loc[:, ~train_df.columns.isin(['Image_path','Amount'])].values
Ytrain = train_df['Amount'].values
Ytrain_strat = pd.qcut(train_df['Amount'].values, q=5, labels=range(0,5))
Xtest = test_df.loc[:, ~test_df.columns.isin(['Image_path'])].values

print("Xtrain: {} \nYtrain: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

Xtrain: (1382, 1691) 
Ytrain: (1382,) 
Xtest: (600, 1691)


0

## Build and validate the model

In [5]:
params = {}
params["objective"] = 'regression'
params["metric"] = 'rmse'
params["boosting"] = 'gbdt'
#params["device_type"] = 'gpu'
params["learning_rate"] = 0.0235
params["lambda_l2"] = 0.008
params["num_leaves"] = 149
params["max_depth"] = 7
params["feature_fraction"] = 0.64
params["bagging_fraction"] = 0.696
params["bagging_freq"] = 3
params["min_data_in_leaf"] = 7
params["verbosity"] = -1
num_rounds = 8000

In [6]:
FOLD = 5
NUM_SEED = 3

np.random.seed(3)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_lgb = np.zeros((Ytrain.shape[0], 1))
y_pred_final_lgb = 0
counter = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain, Ytrain_strat)):
        counter += 1

        train_x, train_y = Xtrain[train], Ytrain[train]
        val_x, val_y = Xtrain[val], Ytrain[val]

        lgtrain = lgb.Dataset(train_x, label=train_y)
        lgvalidation = lgb.Dataset(val_x, label=val_y)

        model = lgb.train(params, lgtrain, num_rounds, 
                          valid_sets=[lgtrain, lgvalidation], 
                          categorical_feature=cat_cols_indices,
                          early_stopping_rounds=100, verbose_eval=200)

        y_pred = model.predict(val_x, num_iteration=model.best_iteration)
        y_pred_meta_lgb[val] += np.array([y_pred]).T
        y_pred_final_lgb += model.predict(Xtest, num_iteration=model.best_iteration)
        
        score = 100 * r2_score(val_y, y_pred)
        oof_score += score
        seed_score += score
        print("\nSeed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_lgb = y_pred_meta_lgb / float(NUM_SEED)
y_pred_final_lgb = y_pred_final_lgb / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[85]	training's rmse: 1360.82	valid_1's rmse: 2470.48

Seed-24 | Fold-0 | OOF Score: 17.576371496698307

Training until validation scores don't improve for 100 rounds
[200]	training's rmse: 653.943	valid_1's rmse: 2418.04
Early stopping, best iteration is:
[103]	training's rmse: 1221.88	valid_1's rmse: 2391.67

Seed-24 | Fold-1 | OOF Score: 19.026593399358084

Training until validation scores don't improve for 100 rounds
[200]	training's rmse: 663.891	valid_1's rmse: 2453.47
Early stopping, best iteration is:
[128]	training's rmse: 1016.35	valid_1's rmse: 2451.37

Seed-24 | Fold-2 | OOF Score: 15.738202994185013

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[69]	training's rmse: 1504.68	valid_1's rmse: 2441.42

Seed-24 | Fold-3 | OOF Score: 21.241816952495498

Training until validation scores don't improve for 100 rounds
Early stopping, be

In [7]:
np.savez_compressed('./LGB_Meta_Features.npz',
                    y_pred_meta_lgb=y_pred_meta_lgb, 
                    oof_score=oof_score,
                    y_pred_final_lgb=y_pred_final_lgb)

## Create submission file

In [8]:
with open("../input/he-ffi-preprocess-data/HE_FFI_Dataset.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
test_df = processed_data['test_df']

del processed_data
gc.collect()

673

In [9]:
test_df1 = pd.read_csv("../input/he-fast-furious-insured/dataset/test.csv")

submit_df = pd.DataFrame()
submit_df['Image_path'] = test_df1['Image_path']
submit_df['Condition'] = test_df['Condition']
submit_df['Amount'] = y_pred_final_lgb

submit_df.to_csv("./LGB_Submission_wo_Adjustment.csv", index=False)
submit_df.head()

,Image_path,Condition,Amount
0,img_4538519.jpg,1.0,3179.437720
1,img_7766002.jpg,1.0,4048.453142
2,img_4637390.jpg,1.0,4310.281828
3,img_4516108.jpg,1.0,4212.673558
4,img_4517008.jpg,1.0,4092.744581


In [10]:
test_df1 = pd.read_csv("../input/he-fast-furious-insured/dataset/test.csv")

submit_df = pd.DataFrame()
submit_df['Image_path'] = test_df1['Image_path']
submit_df['Condition'] = test_df['Condition']
submit_df['Amount'] = y_pred_final_lgb
submit_df.loc[test_df['Condition']==0, 'Amount'] = 0

submit_df.to_csv("./LGB_Submission_w_Adjustment.csv", index=False)
submit_df.head()

,Image_path,Condition,Amount
0,img_4538519.jpg,1.0,3179.437720
1,img_7766002.jpg,1.0,4048.453142
2,img_4637390.jpg,1.0,4310.281828
3,img_4516108.jpg,1.0,4212.673558
4,img_4517008.jpg,1.0,4092.744581
